In [1]:
from obspy import read_inventory, UTCDateTime 
from obspy import read
from obspy.clients.fdsn import Client
from obspy.geodetics import gps2dist_azimuth,kilometers2degrees
from obspy.taup import TauPyModel
from obspy.io.sac import attach_paz

In [2]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime 

### event location research range
locationDic={"minlat":22,"maxlat":26,"minlon":121,"maxlon":122}

### get the event
eveTime=UTCDateTime("2018/02/06 15") ### or UTCDateTime("2019062603")

client = Client("IRIS")
Cata = client.get_events(starttime=eveTime, endtime=eveTime+3600,
                         minlatitude=locationDic['minlat'],maxlatitude=locationDic['maxlat'],
                         minlongitude=locationDic['minlon'], maxlongitude=locationDic['maxlon'],
                         minmag=6,catalog="NEIC PDE")

print(Cata)

1 Event(s) in Catalog:
2018-02-06T15:50:43.300000Z | +24.136, +121.658 | 6.4 Mww


In [3]:
Cata[0]

Event:	2018-02-06T15:50:43.300000Z | +24.136, +121.658 | 6.4 Mww

	            resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?eventid=10639985")
	             event_type: 'earthquake'
	    preferred_origin_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=28369504")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=185558494")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 1 Elements

In [4]:
Cata[0].origins[0]

Origin
	   resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=28369504")
	          time: UTCDateTime(2018, 2, 6, 15, 50, 43, 300000)
	     longitude: 121.658
	      latitude: 24.1359
	         depth: 17000.0
	 creation_info: CreationInfo(author='us')

In [5]:
EVLA=Cata[0].origins[0].latitude
EVLO=Cata[0].origins[0].longitude
EVDP=Cata[0].origins[0].depth/1000
EVTtime=Cata[0].origins[0].time
print(EVTtime)
print(EVLA,EVLO,EVDP)

2018-02-06T15:50:43.300000Z
24.1359 121.658 17.0


In [6]:
#看不懂inventory的可以回去看obspy3.2 get_station那部分，這邊要求所有測站、所有儀器類型、所有分量的資料，且channel要最詳細
inventory = client.get_stations(network="IU", station="*",location="*",channel="*",level="channel")
inv=inventory.copy( )

In [7]:
#去儀器響應
def decon(stf):
    stf.detrend(type='demean')
    stf.detrend(type='linear')
    nqf=stf[0].stats.sampling_rate/2 ###nyquist frequency
    pre_filt = [0.005, 0.008 , nqf-2, nqf] ### zero phase filter band
    stf.remove_response(pre_filt=pre_filt,output='disp') ### 會自動把stf更新
    stf.taper(0.05, type='hann')
    return stf

In [8]:
#要求"Trillium"這種測站，可以對此做更改
def loccode(sta_txt,machine='Trillium'):    
    for txt in sta_txt.channels:
        #print(str(txt))
        if machine in str(txt) :
           # print(txt)
            loc=txt.location_code
            break
            
    
    return loc

In [9]:
#要求"Ttillium"類型的測站，並且只要Z分量的資料，最後轉為SAC檔輸出
for sta_list in inv.select(channel='*Z',time=EVTtime)[0].stations:
    try:
        sta=sta_list.code
        loc=loccode(sta_list)
        print(sta,loc)
        st = client.get_waveforms(network="IU", station=sta,
                          location=loc, channel="BHZ",
                          starttime=EVTtime,endtime=EVTtime + 600,attach_response=True)
        #attach_response=True是要儀器響應，去儀器響應的步驟在上方的def decon
        stf2=st.copy()
        stf=decon(stf2)
        for tr in stf:
            tr.write(tr.id+'.sac',format='SAC')
    except:
        continue

ADK 10
ANTO 10
CHTO 00
COLA 00
DAV 10
DWPF 10
FUNA 10
GNI 10
GUMO 00
HKT 10
JOHN 10
KOWA 00
LCO 10
PTCN 10
RAO 10
SJG 00
TEIG 10
